# Capstone Project - The Battle of the Neighborhoods
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)

## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an **Chinese restaurant** in **Toronto**, Canada, the number of whom is increasing because of the fast growth of Chinese population in the city.

Since there are lots of restaurants in Toronto we will try to detect **locations that are not already crowded with restaurants**. We are also particularly interested in **areas with no or few Chinese restaurants in vicinity**. We would also prefer locations **as close to city center (downtown Toronto) as possible**, assuming that first two conditions are met.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* number of and distance to Chinese restaurants in the neighborhood, if any
* review of the Chinese restaurants in the neighborhood, if any
* distance of neighborhood from city center

We decided to use definition from [this wikipedia link](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* number of restaurants and their type, review and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Toronto neighborhood will be obtained using a csv file input

### Neighborhood in Toronto

We download the neighborhood table from the wikipedia page, the table contains all neighborhoods in Canada, therefore we extract the ones only from Toronto, and attach the coordinates of the neighborhoods using a csv file.

In [6]:
import pandas as pd
import urllib.request
import html5lib
import numpy as np
from bs4 import BeautifulSoup

p_can_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
fp = urllib.request.urlopen(p_can_url)
p_can_html = fp.read()
p_can_html = p_can_html.decode("utf8")
fp.close()
p_can = BeautifulSoup(p_can_html)

df_p_can = []
for tr in p_can.table.find_all('tr'):
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    df_p_can.append(row)
df_p_can=pd.DataFrame(df_p_can,columns=['Postcode','Borough','Neighborhood'])
df_p_can=df_p_can[df_p_can['Borough']!='Not assigned']
df_p_can=df_p_can[df_p_can.Borough.notnull()]
df_p_can['Neighborhood']=df_p_can['Neighborhood'].replace({'\n':''}, regex=True)

df_p_can['Neighborhood'] = np.where(df_p_can['Neighborhood'] == 'Not assigned', df_p_can['Borough'], df_p_can['Neighborhood'])
df_p_can=df_p_can.groupby(['Postcode','Borough'])['Neighborhood'].apply(','.join).reset_index()
coord = pd.read_csv('http://cocl.us/Geospatial_data')
df_coord_can=pd.merge(df_p_can, coord, left_on='Postcode', right_on='Postal Code').drop(['Postal Code'], axis=1)

df_t=df_coord_can[df_coord_can['Borough'].isin(['Downtown Toronto','East Toronto','West Toronto','Central Toronto'])]
df_t.reset_index()
df_t.head()

F:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file F:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


,Postcode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


The data looks good, now we have all the neighborhoods in Toronto and their coordinates.

### Foursquare

Now that we have our neighborhoods data, we can use Foursquare API to get info on (Chinese) restaurants in each neighborhood.

From the API we will get a list of the restaurants, and their types, locations, reviews. And from the list we could feature the neighborhoods with the number of restaurants, the number of Chinese restaurants, average review of the restaurants, and along with their distance to the city center. Using the features, we can cluster the neighborhoods and identify the best (group of) neighborhoods for openning a Chinese restaurant.